In [89]:
import pandas as pd
import altair as alt
import numpy as np

### Notes

1. Data is from https://www.imf.org/external/datamapper/PCPIPCH - download all data as Excel file
2. The data is in old-Excel format. I used LibreOffice Calc to save it as new XLSX format
3. For 2-letter country codes, I used a CSV file from https://pkgstore.datahub.io/core/country-list/data_csv/data/d7c9d7cfb42cb69f4422dec222dbbaa8/data_csv.csv
4. The AU website was misbehaving, so I used an archive.org copy of https://web.archive.org/web/20210424163337/https://au.int/en/member_states/countryprofiles2 for AU member states and regions

### Python modules

Python modules are installed using [conda](https://docs.conda.io/en/latest/). The main modules are pandas, altair, lxml, pyarrow and fastparquet, but my complete `environment.yml` can be found in this notebook's Github repository.

In [101]:
imf_inflation_data = pd.read_excel('data/imf-dm-export-20210525.xlsx', sheet_name='PCPIPCH')
imf_inflation_data.columns = ['country'] + list(imf_inflation_data.columns[1:])  # rename first column to country
imf_inflation_data = imf_inflation_data.drop(0).drop(227).drop(226)  # drop rows not containing data
imf_inflation_data = imf_inflation_data.set_index('country')  # index by country
imf_inflation_data = imf_inflation_data.drop(labels=[year for year in range(2021,2027)], axis=1)  # drop all projected inflation rates
imf_inflation_data

,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,...,11.8,6.4,7.4,4.7,-0.7,4.4,5,0.6,2.3,5.6
Albania,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,...,3.4,2,1.9,1.6,1.9,1.3,2,2,1.4,1.6
Algeria,9.7,14.6,6.6,7.8,6.3,10.4,14,5.9,5.9,9.2,...,4.5,8.9,3.3,2.9,4.8,6.4,5.6,4.3,2,2.4
Angola,46.7,1.4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,13.5,10.3,8.8,7.3,9.2,30.7,29.8,19.6,17.1,22.3
Antigua and Barbuda,19,11.5,4.2,2.3,3.8,1,0.5,3.6,6.8,4.4,...,3.5,3.4,1.1,1.1,1,-0.5,2.4,1.2,1.4,1.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Major advanced economies (G7),12.4,10.3,7.3,4.8,4.6,3.9,2.1,2.7,3.2,4.4,...,2.6,1.9,1.3,1.5,0.3,0.8,1.8,2.1,1.5,0.8
Middle East and Central Asia,11.5,11,9.5,9,7.3,6.1,11.2,14.4,11.4,9.8,...,8.9,8.8,8.3,6.4,5.5,5.7,6.9,9.5,7.4,10.2
Other advanced economies,24.8,16,12.8,11.5,17.3,15.2,5.8,5.3,5.6,6.1,...,3,2,1.6,1.4,0.5,0.9,1.5,1.6,1.1,0.5


In [71]:
country_code_df = pd.read_csv('https://pkgstore.datahub.io/core/country-list/data_csv/data/d7c9d7cfb42cb69f4422dec222dbbaa8/data_csv.csv', index_col=0)
country_code_df

,Code
Name,
Afghanistan,AF
Åland Islands,AX
Albania,AL
Algeria,DZ
American Samoa,AS
...,...
Wallis and Futuna,WF
Western Sahara,EH
Yemen,YE


In [11]:
tables = pd.read_html('https://web.archive.org/web/20210424163337/https://au.int/en/member_states/countryprofiles2', header=0)

In [22]:
central_africa_df = tables[2]
east_africa_df = tables[3]
north_africa_df = tables[4]
south_africa_df = tables[5]
west_africa_df = tables[6]
all_africa_df = pd.concat([central_africa_df, east_africa_df, north_africa_df, south_africa_df, west_africa_df])

In [85]:
africa_country_to_imf_name = {
    'Congo Republic': 'Congo, Republic of ',
    'DR Congo': 'Congo, Dem. Rep. of the',
    'São Tomé and Príncipe': 'Sao Tome and Principe',
    'Tanzania': 'Tanzania, United Republic of',
    'Eswatini': 'Swaziland',
    'Cabo Verde': 'Cape Verde',
    'Côte d’Ivoire': "Côte d'Ivoire",
    'South Sudan': 'South Sudan, Republic of',
    'Gambia': 'Gambia, The'
}

africa_country_to_name = {
    'Congo Republic': 'Congo',
    'DR Congo': 'Congo, the Democratic Republic of the',
    'São Tomé and Príncipe': 'Sao Tome and Principe',
    'Tanzania': 'Tanzania, United Republic of',
    'Eswatini': 'Swaziland',
    'Cabo Verde': 'Cape Verde',
    'Côte d’Ivoire': "Côte d'Ivoire",
    'South Sudan': 'South Sudan, Republic of',
    'Gambia': 'Gambia, The'
}
for country in all_africa_df.Abbreviation:
    if country not in imf_inflation_data.index:
        if country not in africa_country_to_name:
            print(country)

Somalia
Sahrawi Republic


In [114]:
data = {}
for country in all_africa_df.Abbreviation:
    if country in central_africa_df.Abbreviation.values:
        region = 'central'
    elif country in east_africa_df.Abbreviation.values:
        region = 'east'
    elif country in north_africa_df.Abbreviation.values:
        region = 'north'
    elif country in south_africa_df.Abbreviation.values:
        region = 'south'
    elif country in west_africa_df.Abbreviation.values:
        region = 'west'
    else:
        raise ValueError('Unknown country: ' + country)

    if country in ('Sahrawi Republic', 'Somalia'):
        # we don't have info from IMF for these two countries
        continue
    elif country in country_code_df.index:
        lookup_name = country
    elif country in africa_country_to_name:
        lookup_name = africa_country_to_name[country]
    else:
        raise ValueError('Cannot lookup country: ' + country)
    
    abbrev = country_code_df.loc[lookup_name, 'Code']
    
    if country in imf_inflation_data.index:
        lookup_name = country
    elif country in africa_country_to_imf_name:
        lookup_name = africa_country_to_imf_name[country]
    
    inflation_data = list(imf_inflation_data.loc[lookup_name])
    data[country] = [region, abbrev] + inflation_data
inflation_df = pd.DataFrame.from_dict(data, orient='index', columns=['region', 'abbreviation'] + list(imf_inflation_data.columns)).replace('no data', np.nan)
inflation_df.index.name = 'country'
inflation_df

,region,abbreviation,1980,1981,1982,1983,1984,1985,1986,1987,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
country,,,,,,,,,,,,,,,,,,,,,
Burundi,central,BI,1.2,12.2,5.9,8.2,14.3,3.8,1.7,7.1,...,9.6,18.2,7.9,4.4,5.6,5.5,1.6,-4.0,-0.7,7.3
Cameroon,central,CM,7.7,7.5,15.3,20.5,12.1,4.2,4.3,2.8,...,2.9,2.4,2.1,1.9,2.7,0.9,0.6,1.1,2.5,2.8
Central African Republic,central,CF,13.3,14.7,13.2,14.6,2.6,10.5,2.4,-7.0,...,1.2,5.5,7.0,14.9,1.4,4.9,4.2,1.6,2.7,2.3
Chad,central,TD,8.6,8.1,6.4,7.9,20.3,5.1,-13.1,-2.7,...,2.0,7.5,0.2,1.7,4.8,-1.6,-0.9,4.0,-1.0,3.5
Congo Republic,central,CG,7.3,0.8,3.5,3.5,3.5,3.5,3.5,3.5,...,1.8,5.0,4.6,0.9,3.2,3.2,0.4,1.2,2.2,2.4
DR Congo,central,CD,40.0,34.9,37.1,76.7,52.6,23.5,46.7,90.4,...,14.9,0.9,0.9,1.2,0.7,3.2,35.8,29.3,4.7,11.3
Equatorial Guinea,central,GQ,0.0,16.7,38.3,60.1,59.5,84.0,-17.6,-13.2,...,4.8,3.4,3.2,4.3,1.7,1.4,0.7,1.3,1.2,4.8
Gabon,central,GA,12.3,8.6,16.7,10.4,5.9,7.3,6.4,-1.0,...,1.3,2.7,0.5,4.5,-0.1,2.1,2.7,4.8,2.0,1.3
São Tomé and Príncipe,central,ST,0.2,11.5,0.4,7.0,3.2,0.1,13.9,25.0,...,14.3,10.6,8.1,7.0,6.1,5.4,5.7,7.9,7.7,9.8


In [145]:
# write the data in Excel, CSV and Parquet formats
inflation_df.to_excel('data/africa_inflation_data.xlsx', sheet_name='Inflation')
inflation_df.to_csv('data/africa_inflation_data.csv')
inflation_df.columns = [str(column) for column in inflation_df.columns]
inflation_df.to_parquet('data/africa_inflation_data.parquet')
# Parquet is a format that retains the complete DataFrame structure and can be read back quite simply:
# pd.read_parquet('data/africa_inflation_data.parquet')

### Long form and plotting

When plotting data with e.g. Altair, the data should be in [*long form*](https://altair-viz.github.io/user_guide/data.html#long-form-vs-wide-form-data), that is, one row per observation with metadata (e.g. year names) as values in the dataframe.

The next code reformats the data into long form and then illustrates plotting with Altair

In [139]:
inflation_longform_df = inflation_df.reset_index().melt(['country', 'region', 'abbreviation'], var_name=['year'], value_name='inflation')
inflation_longform_df

,country,region,abbreviation,year,inflation
0,Burundi,central,BI,1980,1.2
1,Cameroon,central,CM,1980,7.7
2,Central African Republic,central,CF,1980,13.3
3,Chad,central,TD,1980,8.6
4,Congo Republic,central,CG,1980,7.3
...,...,...,...,...,...
2168,Niger,west,NE,2020,2.8
2169,Nigeria,west,NG,2020,13.2
2170,Senegal,west,SN,2020,2.5
2171,Sierra Leone,west,SL,2020,15.7


In [140]:
alt.Chart(inflation_longform_df[inflation_longform_df.country.isin(['South Africa', 'Kenya'])]).mark_line().encode(x=alt.X('year:O', title='Year'),
                                                                                                                   y=alt.Y('inflation', title='Inflation (%)'),
                                                                                                                   color='country'
                                                                                                                  )

alt.Chart(...)

,region,abbreviation,1980,1981,1982,1983,1984,1985,1986,1987,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
country,,,,,,,,,,,,,,,,,,,,,
Burundi,central,BI,1.2,12.2,5.9,8.2,14.3,3.8,1.7,7.1,...,9.6,18.2,7.9,4.4,5.6,5.5,1.6,-4.0,-0.7,7.3
Cameroon,central,CM,7.7,7.5,15.3,20.5,12.1,4.2,4.3,2.8,...,2.9,2.4,2.1,1.9,2.7,0.9,0.6,1.1,2.5,2.8
Central African Republic,central,CF,13.3,14.7,13.2,14.6,2.6,10.5,2.4,-7.0,...,1.2,5.5,7.0,14.9,1.4,4.9,4.2,1.6,2.7,2.3
Chad,central,TD,8.6,8.1,6.4,7.9,20.3,5.1,-13.1,-2.7,...,2.0,7.5,0.2,1.7,4.8,-1.6,-0.9,4.0,-1.0,3.5
Congo Republic,central,CG,7.3,0.8,3.5,3.5,3.5,3.5,3.5,3.5,...,1.8,5.0,4.6,0.9,3.2,3.2,0.4,1.2,2.2,2.4
DR Congo,central,CD,40.0,34.9,37.1,76.7,52.6,23.5,46.7,90.4,...,14.9,0.9,0.9,1.2,0.7,3.2,35.8,29.3,4.7,11.3
Equatorial Guinea,central,GQ,0.0,16.7,38.3,60.1,59.5,84.0,-17.6,-13.2,...,4.8,3.4,3.2,4.3,1.7,1.4,0.7,1.3,1.2,4.8
Gabon,central,GA,12.3,8.6,16.7,10.4,5.9,7.3,6.4,-1.0,...,1.3,2.7,0.5,4.5,-0.1,2.1,2.7,4.8,2.0,1.3
São Tomé and Príncipe,central,ST,0.2,11.5,0.4,7.0,3.2,0.1,13.9,25.0,...,14.3,10.6,8.1,7.0,6.1,5.4,5.7,7.9,7.7,9.8
